# Yale Dataset
the dataset was already skullstripped and coregistered onto SRI24\
To-do:
- n4 bias correction
- percentile normalization

In [1]:
from tqdm import tqdm
from datetime import datetime

import nibabel as nib

#import ants

import SimpleITK as sitk
from pathlib import Path
from intensity_normalization.normalize.zscore import ZScoreNormalize

#import intensity_normalization

import numpy as np

import os

In [2]:
path_to_yale_dataset = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/Pretreat-MetsToBrain-Masks")
path_to_output = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/CIA metstobrain/yale_dataset_normalized")

In [3]:
# n4 bias correction

def n4_bias_correction(image, path_to_output):
    """
    Applies N4 bias field correction to the given image and saves the corrected image to the specified output path.

    Parameters:
        image (sitk.Image): The input image to be corrected.
        path_to_output (str or pathlib.Path): The path to save the corrected image.

    Returns:
        None
    """

    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    image_corrected = corrector.Execute(image)

    sitk.WriteImage(image_corrected, str(path_to_output))

def percentile_normalize(image: np.ndarray, lower_percentile: float = 0.0, upper_percentile: float = 100.0, lower_limit: float = 0, upper_limit: float = 1):
    """
    Normalizes an image using percentile-based mapping.

    Args:
        image (numpy.ndarray): The input image.
        lower_percentile (float, optional): The lower percentile for mapping. Defaults to 0.0.
        upper_percentile (float, optional): The upper percentile for mapping. Defaults to 100.0.
        lower_limit (float, optional): The lower limit for normalized values. Defaults to 0.
        upper_limit (float, optional): The upper limit for normalized values. Defaults to 1.

    Returns:
        numpy.ndarray: The percentile-normalized image.

    Description:
        This function takes an image as input and normalizes it using percentile-based mapping. It calculates the lower
        and upper values of the image based on the provided percentiles. The normalized image is then calculated by
        subtracting the lower value from each pixel value, dividing the result by the difference between the upper and
        lower values, and clipping the values between 0 and 1. Finally, the normalized image is scaled to the specified
        lower limit and upper limit.

    Example:
        >>> image = np.array([10, 20, 30, 40, 50])
        >>> percentile_normalize(image, lower_percentile=10, upper_percentile=90)
        array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444])
    """


    lower_value = np.percentile(image, lower_percentile)
    upper_value = np.percentile(image, upper_percentile)

    normalized_image = np.clip(
        (image - lower_value) / (upper_value - lower_value), 0, 1
    )
    normalized_image = (
        normalized_image * (upper_limit - lower_limit) + lower_limit
    )

    normalized_upper_value = np.percentile(normalized_image, upper_percentile)
    normalized_lower_value = np.percentile(normalized_image, lower_percentile)

    if normalized_upper_value != 1 or normalized_lower_value != 0:
        print(f"Percentile normalization failed. Upper value: {normalized_upper_value}, lower value: {normalized_lower_value}")

    return normalized_image

In [5]:
patients = [patient for patient in os.listdir(path_to_yale_dataset) if os.path.isdir(os.path.join(path_to_yale_dataset, patient))]
print(f"found {len(patients)} patients")

already_norm_patients = [patient for patient in os.listdir(path_to_output) if os.path.isdir(os.path.join(path_to_output, patient))]

for patient in already_norm_patients:
    # check if 4 n4 and percentile normalized files exist
    path_to_patient_folder = Path(path_to_output) / Path(patient)
    nifti_files = [path for path in os.listdir(path_to_patient_folder) if ".nii.gz" in path]
    if len(nifti_files) == 8:
        patients.remove(patient)
    else:
        # remove all files in patient folder
        for file in os.listdir(path_to_patient_folder):
            os.remove(os.path.join(path_to_patient_folder, file))

print(f"found {len(patients)} patients to normalize")

for patient in tqdm(patients):

    print(f"currently normalizing patient {patient}")

    path_to_patient_files = Path(path_to_yale_dataset) / Path(patient)

    raw_files = [file for file in os.listdir(path_to_patient_files) if ".nii.gz" in file]

    # create subfolder for each patient
    path_to_patient_folder = Path(path_to_output) / Path(patient)
    path_to_patient_folder.mkdir(parents=True, exist_ok=True)

    for raw_file in raw_files:

        if "seg" in raw_file:
            continue

        path_to_file = Path(path_to_patient_files) / Path(raw_file)
        print(f"starting n4 bias correction for {raw_file}")

        image = sitk.ReadImage(str(path_to_file), imageIO="NiftiImageIO")
        image = sitk.Cast(image, sitk.sitkFloat32)

        n4_file_name = f"n4_{raw_file}"
        path_to_n4_file = Path(path_to_patient_folder) / Path(n4_file_name)

        n4_bias_correction(image, path_to_n4_file)

        print(f"finished n4 bias correction for {raw_file}")

        # percentile normalize
        print(f"starting percentile correction for {raw_file}")
        image = sitk.ReadImage(str(path_to_n4_file))

        normalized_file_name = f"n4_normalized_{raw_file}"
        path_to_normalized_file = Path(path_to_patient_folder) / Path(normalized_file_name)

        # Get the original orientation information
        original_direction = image.GetDirection()
        original_origin = image.GetOrigin()
        original_spacing = image.GetSpacing()

        image_array = sitk.GetArrayFromImage(image)
        image_array = percentile_normalize(image_array, lower_percentile=0.1, upper_percentile=99.9, lower_limit=0, upper_limit=1)

        image = sitk.GetImageFromArray(image_array)

        # Restore the original orientation information
        image.SetDirection(original_direction)
        image.SetOrigin(original_origin)
        image.SetSpacing(original_spacing)

        sitk.WriteImage(image, str(path_to_normalized_file))

        print(f"finished percentile normalization for {raw_file}")

found 200 patients
found 195 patients to normalize


  0%|          | 0/195 [00:00<?, ?it/s]

currently normalizing patient BraTS-MET-00119-000
starting n4 bias correction for BraTS-MET-00119-000-t1c.nii.gz
finished n4 bias correction for BraTS-MET-00119-000-t1c.nii.gz
starting percentile correction for BraTS-MET-00119-000-t1c.nii.gz
finished percentile normalization for BraTS-MET-00119-000-t1c.nii.gz
starting n4 bias correction for BraTS-MET-00119-000-t2w.nii.gz
finished n4 bias correction for BraTS-MET-00119-000-t2w.nii.gz
starting percentile correction for BraTS-MET-00119-000-t2w.nii.gz
finished percentile normalization for BraTS-MET-00119-000-t2w.nii.gz
starting n4 bias correction for BraTS-MET-00119-000-t1n.nii.gz
finished n4 bias correction for BraTS-MET-00119-000-t1n.nii.gz
starting percentile correction for BraTS-MET-00119-000-t1n.nii.gz
finished percentile normalization for BraTS-MET-00119-000-t1n.nii.gz
starting n4 bias correction for BraTS-MET-00119-000-t2f.nii.gz
finished n4 bias correction for BraTS-MET-00119-000-t2f.nii.gz
starting percentile correction for BraTS-

  1%|          | 1/195 [01:12<3:54:05, 72.40s/it]

finished percentile normalization for BraTS-MET-00119-000-t2f.nii.gz
currently normalizing patient BraTS-MET-00131-000
starting n4 bias correction for BraTS-MET-00131-000-t1n.nii.gz
finished n4 bias correction for BraTS-MET-00131-000-t1n.nii.gz
starting percentile correction for BraTS-MET-00131-000-t1n.nii.gz
finished percentile normalization for BraTS-MET-00131-000-t1n.nii.gz
starting n4 bias correction for BraTS-MET-00131-000-t2f.nii.gz
finished n4 bias correction for BraTS-MET-00131-000-t2f.nii.gz
starting percentile correction for BraTS-MET-00131-000-t2f.nii.gz
finished percentile normalization for BraTS-MET-00131-000-t2f.nii.gz
starting n4 bias correction for BraTS-MET-00131-000-t2w.nii.gz
finished n4 bias correction for BraTS-MET-00131-000-t2w.nii.gz
starting percentile correction for BraTS-MET-00131-000-t2w.nii.gz
finished percentile normalization for BraTS-MET-00131-000-t2w.nii.gz
starting n4 bias correction for BraTS-MET-00131-000-t1c.nii.gz
finished n4 bias correction for Bra

...

finished percentile normalization for BraTS-MET-00276-000-t2w.nii.gz
currently normalizing patient BraTS-MET-00262-000
starting n4 bias correction for BraTS-MET-00262-000-t2f.nii.gz
finished n4 bias correction for BraTS-MET-00262-000-t2f.nii.gz
starting percentile correction for BraTS-MET-00262-000-t2f.nii.gz
finished percentile normalization for BraTS-MET-00262-000-t2f.nii.gz
starting n4 bias correction for BraTS-MET-00262-000-t1n.nii.gz
finished n4 bias correction for BraTS-MET-00262-000-t1n.nii.gz
starting percentile correction for BraTS-MET-00262-000-t1n.nii.gz
finished percentile normalization for BraTS-MET-00262-000-t1n.nii.gz
starting n4 bias correction for BraTS-MET-00262-000-t1c.nii.gz
finished n4 bias correction for BraTS-MET-00262-000-t1c.nii.gz
starting percentile correction for BraTS-MET-00262-000-t1c.nii.gz
finished percentile normalization for BraTS-MET-00262-000-t1c.nii.gz
starting n4 bias correction for BraTS-MET-00262-000-t2w.nii.gz
finished n4 bias correction for Bra

 99%|█████████▉| 194/195 [4:11:49<01:12, 72.95s/it]

finished percentile normalization for BraTS-MET-00262-000-t2w.nii.gz
currently normalizing patient BraTS-MET-00289-000
starting n4 bias correction for BraTS-MET-00289-000-t1n.nii.gz
finished n4 bias correction for BraTS-MET-00289-000-t1n.nii.gz
starting percentile correction for BraTS-MET-00289-000-t1n.nii.gz
finished percentile normalization for BraTS-MET-00289-000-t1n.nii.gz
starting n4 bias correction for BraTS-MET-00289-000-t2f.nii.gz
finished n4 bias correction for BraTS-MET-00289-000-t2f.nii.gz
starting percentile correction for BraTS-MET-00289-000-t2f.nii.gz
finished percentile normalization for BraTS-MET-00289-000-t2f.nii.gz
starting n4 bias correction for BraTS-MET-00289-000-t1c.nii.gz
finished n4 bias correction for BraTS-MET-00289-000-t1c.nii.gz
starting percentile correction for BraTS-MET-00289-000-t1c.nii.gz
Percentile normalization failed. Upper value: 0.9999998809099198, lower value: 0.0
finished percentile normalization for BraTS-MET-00289-000-t1c.nii.gz
starting n4 bia

100%|██████████| 195/195 [4:12:56<00:00, 77.83s/it]

finished percentile normalization for BraTS-MET-00289-000-t2w.nii.gz
